In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
#from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
#from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

import keras 
import tensorflow as tf
from keras.layers import Dense, Flatten,Conv1D, MaxPooling1D, Dropout, Activation,merge,Lambda,concatenate
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,multiply
from keras.layers.embeddings import Embedding
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer,one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

ModuleNotFoundError: No module named 'keras'

In [2]:
#data_train = pd.read_csv('/Users/s0c02nj/Desktop/NER and Innoplexus-analytics vidhya/train.csv')
#data_test =  pd.read_csv('/Users/s0c02nj/Desktop/NER and Innoplexus-analytics vidhya/test 4.csv')
#data_sub = pd.read_csv('/Users/s0c02nj/Desktop/NER and Innoplexus-analytics vidhya/sample_submission 2.csv')

In [3]:
data = pd.read_csv("/Users/s0c02nj/Desktop/Personal Edu doc/NER-work/ner_dataset.csv", encoding="latin1")

In [4]:
#list(data_train[data_train['Sent_ID']==1]['Word'])
data.head()
data = data.fillna(method="ffill")

In [5]:
#data.tail(1)
data['Word']=data['Word'].str.lower()

In [6]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

31817

In [7]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [9]:
getter = SentenceGetter(data)
sentences = getter.sentences

In [10]:
max_len = 75
max_len_char = 10

In [11]:
#Creating the Dictionary and Inverse
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0

idx2word = {i: w for w, i in word2idx.items()}
tag2idx  = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [12]:
#print(word2idx["Obama"])
#print(tag2idx["B-geo"])
#Getting the sequences at word level
X_word = [[word2idx[w[0]] for w in s] for s in sentences]
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')


#### Character Sequence

In [13]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

72


In [43]:
char2idx = {c: i+2 for i,c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [44]:
sentences[0][1][0][0]

'o'

In [45]:
#Defining the character encoding

X_char = []
for sentence in tqdm(sentences):
    sent_seq = []
    #To give an upper bound on the maximum length of the word sequence for sentence
    for i in range(max_len):
        word_seq = []
        #to give an upper bound on the maximum length of words to consider
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:  #exception will be there when there will not be any word for the length and will be padded 0
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))


100%|██████████| 47959/47959 [00:24<00:00, 1967.60it/s]


In [55]:
#X_char[0]

In [56]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [61]:
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

In [62]:
from sklearn.model_selection import train_test_split

X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

+ input_dim: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.


+ output_dim: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.


+ input_length: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.

In [95]:
# input and embedding for words
word_in =  Input(shape=(max_len,))
emb_word = Embedding(input_dim= n_words+2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

#input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))

#Character embedding
emb_char = TimeDistributed(Embedding(input_dim=n_chars +2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)

# #character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# #main LSTM
x = concatenate([emb_word, char_enc])

#x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)

out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(main_lstm)


model = Model([word_in, char_in], out)

In [97]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_40 (InputLayer)           (None, 75, 10)       0                                            
__________________________________________________________________________________________________
input_39 (InputLayer)           (None, 75)           0                                            
__________________________________________________________________________________________________
time_distributed_23 (TimeDistri (None, 75, 10, 10)   740         input_40[0][0]                   
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 75, 20)       636380      input_39[0][0]                   
__________________________________________________________________________________________________
time_distr

In [98]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])


In [99]:
history = model.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                     np.array(y_tr).reshape(len(y_tr), max_len, 1),
                     batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Epoch 1/10


InvalidArgumentError: Incompatible shapes: [2400] vs. [32,75]
	 [[{{node metrics/acc/Equal}} = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics/acc/Reshape, metrics/acc/Cast)]]